In [1]:
%matplotlib qt
import src
from src.simul_sweep import SimulSweep
from src.daq_driver import Daq
from src.base_sweep import BaseSweep
from src.sweep0d import Sweep0D
from src.sweep1d import Sweep1D
from src.sweep2d import Sweep2D
from src.sweep_ips import SweepIPS
from src.sweep_queue import SweepQueue, DatabaseEntry
from src.util import init_database
from src.tracking import *
import nidaqmx
import time, os
import numpy as np
import qcodes as qc
from src.util import connect_to_station, connect_station_instruments
from qcodes import initialise_or_create_database_at, load_by_run_spec
from qcodes import Station
from qcodes.instrument_drivers.Lakeshore.Model_372 import Model_372
from qcodes.instrument_drivers.american_magnetics.AMI430 import AMI430
from qcodes.instrument_drivers.stanford_research.SR860 import SR860
from qcodes_contrib_drivers.drivers.Oxford.IPS120 import OxfordInstruments_IPS120
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.data_set import load_by_run_spec
from qcodes.dataset.data_export import get_data_by_id
from qcodes.instrument_drivers.stanford_research.SR830 import SR830

C:\Users\nanouser\Anaconda3\envs\qcodes\lib\site-packages\visa.py:23: FutureWarning: The visa module provided by PyVISA is being deprecated. You can replace `import visa` by `import pyvisa as visa` to achieve the same effect.

The reason for the deprecation is the possible conflict with the visa package provided by the https://github.com/visa-sdk/visa-python which can result in hard to debug situations.
  FutureWarning,


In [2]:
station = Station(config_file='cfg/dacen_21_1_20.station.yaml')

In [3]:
station.snapshot()

{'instruments': {},
 'parameters': {},
 'components': {},
 'config': {'instruments': ordereddict([('daq', ordereddict([('address', 'Dev1'), ('enable_forced_reconnect', True), ('type', 'src.daq_driver.Daq')])), ('keithley_bg', ordereddict([('address', 'USB0::0x05E6::0x2450::04428921::INSTR'), ('enable_forced_reconnect', True), ('type', 'qcodes.instrument_drivers.tektronix.Keithley_2450.Keithley2450')])), ('keithley_sg', ordereddict([('address', 'USB0::0x05E6::0x2450::04453530::INSTR'), ('enable_forced_reconnect', True), ('type', 'qcodes.instrument_drivers.tektronix.Keithley_2450.Keithley2450')])), ('keithley_tg', ordereddict([('address', 'USB0::0x05E6::0x2450::04428406::INSTR'), ('enable_forced_reconnect', True), ('type', 'qcodes.instrument_drivers.tektronix.Keithley_2450.Keithley2450')])), ('sr830', ordereddict([('address', 'GPIB0::8::INSTR'), ('enable_forced_reconnect', True), ('type', 'qcodes.instrument_drivers.stanford_research.SR830.SR830')])), ('sr860', ordereddict([('address', 'U

In [4]:
station.snapshot()['config']['instruments'].items()

In [5]:
instruments = {}
for (name, other) in station.snapshot()['config']['instruments'].items():
    instruments[name] = station.load_instrument(name)

[daq_ao0(DaqAOChannel)] Snapshot: Could not update parameter: gain_factor
[daq_ao0(DaqAOChannel)] Snapshot: Could not update parameter: impedance
[daq_ao0(DaqAOChannel)] Snapshot: Could not update parameter: value
[daq_ao1(DaqAOChannel)] Snapshot: Could not update parameter: gain_factor
[daq_ao1(DaqAOChannel)] Snapshot: Could not update parameter: impedance
[daq_ao1(DaqAOChannel)] Snapshot: Could not update parameter: value
[daq_ao2(DaqAOChannel)] Snapshot: Could not update parameter: gain_factor
[daq_ao2(DaqAOChannel)] Snapshot: Could not update parameter: impedance
[daq_ao2(DaqAOChannel)] Snapshot: Could not update parameter: value
[daq_ao3(DaqAOChannel)] Snapshot: Could not update parameter: gain_factor
[daq_ao3(DaqAOChannel)] Snapshot: Could not update parameter: impedance
[daq_ao3(DaqAOChannel)] Snapshot: Could not update parameter: value
[daq_ai0(DaqAIChannel)] Snapshot: Could not update parameter: gain_factor
[daq_ai0(DaqAIChannel)] Snapshot: Could not update parameter: impedanc

Connected to: NI DAQ USB-6259 (BNC) (Dev1) in 0.15s


[daq_ai5(DaqAIChannel)] Snapshot: Could not update parameter: impedance
[daq_ai5(DaqAIChannel)] Snapshot: Could not update parameter: value
[daq_ai6(DaqAIChannel)] Snapshot: Could not update parameter: gain_factor
[daq_ai6(DaqAIChannel)] Snapshot: Could not update parameter: impedance
[daq_ai6(DaqAIChannel)] Snapshot: Could not update parameter: value
[daq_ai7(DaqAIChannel)] Snapshot: Could not update parameter: gain_factor
[daq_ai7(DaqAIChannel)] Snapshot: Could not update parameter: impedance
[daq_ai7(DaqAIChannel)] Snapshot: Could not update parameter: value
[daq_ai16(DaqAIChannel)] Snapshot: Could not update parameter: gain_factor
[daq_ai16(DaqAIChannel)] Snapshot: Could not update parameter: impedance
[daq_ai16(DaqAIChannel)] Snapshot: Could not update parameter: value
[daq_ai17(DaqAIChannel)] Snapshot: Could not update parameter: gain_factor
[daq_ai17(DaqAIChannel)] Snapshot: Could not update parameter: impedance
[daq_ai17(DaqAIChannel)] Snapshot: Could not update parameter: valu

Connected to: KEITHLEY INSTRUMENTS 2450 (serial:04428921, firmware:1.6.7c) in 0.04s


[keithley_bg_source(Source2450)] Snapshot: Could not update parameter: user_delay
[keithley_bg_source(Source2450)] Snapshot: Could not update parameter: sweep_axis
[keithley_bg_source(Source2450)] Snapshot: Could not update parameter: user_delay
[keithley_bg_sense(Sense2450)] Snapshot: Could not update parameter: user_delay
[keithley_bg_sense(Sense2450)] Snapshot: Could not update parameter: user_delay
[keithley_bg_sense(Sense2450)] Snapshot: Could not update parameter: user_delay
[keithley_sg_source(Source2450)] Snapshot: Could not update parameter: sweep_axis


Connected to: KEITHLEY INSTRUMENTS 2450 (serial:04453530, firmware:1.6.7c) in 0.02s


[keithley_sg_source(Source2450)] Snapshot: Could not update parameter: user_delay
[keithley_sg_source(Source2450)] Snapshot: Could not update parameter: sweep_axis
[keithley_sg_source(Source2450)] Snapshot: Could not update parameter: user_delay
[keithley_sg_sense(Sense2450)] Snapshot: Could not update parameter: user_delay
[keithley_sg_sense(Sense2450)] Snapshot: Could not update parameter: user_delay
[keithley_sg_sense(Sense2450)] Snapshot: Could not update parameter: user_delay
[keithley_tg_source(Source2450)] Snapshot: Could not update parameter: sweep_axis


Connected to: KEITHLEY INSTRUMENTS 2450 (serial:04428406, firmware:1.6.7c) in 0.02s


[keithley_tg_source(Source2450)] Snapshot: Could not update parameter: user_delay
[keithley_tg_source(Source2450)] Snapshot: Could not update parameter: sweep_axis
[keithley_tg_source(Source2450)] Snapshot: Could not update parameter: user_delay
[keithley_tg_sense(Sense2450)] Snapshot: Could not update parameter: user_delay
[keithley_tg_sense(Sense2450)] Snapshot: Could not update parameter: user_delay
[keithley_tg_sense(Sense2450)] Snapshot: Could not update parameter: user_delay


Connected to: Stanford_Research_Systems SR830 (serial:s/n31208, firmware:ver1.07) in 0.07s
Connected to: Stanford_Research_Systems SR860 (serial:003594, firmware:V1.51) in 0.06s


In [7]:
print(instruments)
daq = instruments['daq']
keithley_bg = instruments['keithley_bg']
keithley_tg = instruments['keithley_tg']
keithley_sg = instruments['keithley_sg']
#ls340 = instruments['ls340']
sr830 = instruments['sr830']
sr860 = instruments['sr860']

{'daq': <Daq: daq>, 'keithley_bg': <Keithley2450: keithley_bg>, 'keithley_sg': <Keithley2450: keithley_sg>, 'keithley_tg': <Keithley2450: keithley_tg>, 'sr830': <SR830: sr830>, 'sr860': <SR860: sr860>}


In [8]:
daq.ai0.voltage.label = 'Rxx_T5-T6'
daq.ai1.voltage.label = 'Rxx_p_T5-T6'
daq.ai2.voltage.label = 'Rxx_T2-T3'
daq.ai3.voltage.label = 'Rxx_p_T2-T3'
daq.ai4.voltage.label = 'Rxx_B1-B2'
daq.ai5.voltage.label = 'Rxx_p_B1-B2'
keithley_bg.sense.current.label = 'I_bg'
keithley_tg.sense.current.label = 'I_tg'
sr830.X.label = 'Rxy_T2-T5'
sr830.P.label = 'Rxy_p_T2-T5'
sr860.X.label = 'I_s'
sr860.P.label = 'I_s_p'

In [9]:
db = os.environ['MeasureItHome']+'\\Databases\\dw_14-DtBLG.db'

## Initialize the magnet driver

In [10]:
magnet = OxfordInstruments_IPS120('magnet', 'GPIB0::25::INSTR', use_gpib=True)

## Let's make sure we're connected and everything is as we expect

In [11]:
magnet.identify()

'IPS120-10  Version 3.07  (c) OXFORD 1996'

In [12]:
magnet.examine()

System Status: 
Normal
Activity: 


KeyError: (4, 'getting magnet_activity')

## Set the control status 
#### 0 : Local and locked
#### 1 : Remote and locked
#### 2 : Local and unlocked
#### 3 : Remote and unlocked

In [12]:
magnet.remote_status(3)

### We can check and set the field setpoints and the sweeprate of the field

In [ ]:
magnet.field_setpoint()

In [ ]:
magnet.field_setpoint(0)

In [ ]:
magnet.sweeprate_field()

In [ ]:
magnet.sweeprate_field(0.3)

## Check (and set) the activity status
#### 0 : Hold
#### 1 : Goto zero
#### 2 : Goto setpoint

In [ ]:
magnet.activity()

## Turn the switch heater on to enable the field to be ramped

In [ ]:
magnet.heater_on()

## Now, we can run to the field, either by calling 'run_to_field', or setting the activity to 'goto setpoint' 

In [ ]:
magnet.run_to_field(12)

## If we want to stay at field to do measurements, hold the field in place by setting persistent mode

In [ ]:
magnet.set_persistent()

#### You can check that we're in persistent mode by checking the persistent field (field in the magnet) and the field (which is the current going through the leads- should be 0)

In [ ]:
magnet.persistent_field()

In [ ]:
magnet.field()

## Once done, leave persistent mode so we can ramp the field back to zero

In [ ]:
magnet.leave_persistent_mode()

## Once we are done, we can go back to 0 either by 'run_to_field' or setting activity to 'goto zero'

In [ ]:
magnet.run_to_field(0)

## Finally, switch the heater off and set us to 'hold'

In [ ]:
magnet.heater_off()
magnet.activity(0)

## Double check to make sure the field and current are 0, and it is set to hold, and you can turn the IPS power off

In [17]:
keithley_tg.output_enabled(1)
keithley_bg.output_enabled(1)
keithley_sg.output_enabled(1)

In [ ]:
done_with_sweeps=False
print(done_with_sweeps)
def iterate():
    global done_with_sweeps
    done_with_sweeps=True

iterate()
print(done_with_sweeps)

In [ ]:
keithley_tg.source.voltage()

In [ ]:
magnet.run_to_field(0)

In [ ]:
keithley_tg.source_voltage(0)

In [ ]:
ls340.A.temperature()

In [ ]:
sweep0 = Sweep0D(inter_delay=0.5)
sweep0.follow_param(magnet.field)

In [ ]:
sweep0.start()

In [ ]:
magnet.field.get()

In [ ]:
sweep.instrument

In [27]:
magnet.heater_off()

Setting switch heater... (wait 40s)


In [28]:
magnet.activity(0)

In [20]:
magnet.leave_persistent_mode()

Setting switch heater... (wait 40s)


In [15]:
magnet.set_persistent()

Setting switch heater... (wait 40s)
Waiting for the switch heater to become superconducting


In [ ]:
magnet.field_setpoint(0.1)

In [ ]:
magnet.mode2()

In [ ]:
magnet.persistent_field()

In [25]:
sweep = SweepIPS(magnet, 0, inter_delay=0.25, save_data=False, persistent_magnet=False, plot_data = True)
#sweep.follow_param(ls340.A.temperature)

In [26]:
sweep.start()

Checking the status of the magnet and switch heater.
Setting switch heater... (wait 40s)
Done with the sweep, t=1301.922000000006 s
Done with the sweep!


In [18]:
sweep.plotter.data_queue

deque([])

In [19]:
sweep.stop()

Sweep not currently running. Nothing to stop.


In [19]:
sweep.kill()

In [18]:
ramp_sg = Sweep1D(keithley_sg.source.voltage, 0, 10, -.1, inter_delay=0.25, save_data=False, x_axis_time = False)
ramp_sg.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage, sr830.X, sr830.P, sr860.X, sr860.P)

In [19]:
ramp_sg.start()

Ramping to our starting setpoint value of 0 V
Done ramping voltage to 0
Sweeping voltage to 10 V


In [22]:
sweep_queue = SweepQueue(inter_delay = 1)
exp_name = '210203-Landau_fan_unknown_temp_Vsg=10V'

Vtg_min, Vtg_max = -8., 8.
Vtg_step_slow = 0.05
Vtg_step_fast = 0.2

Vbg_min, Vbg_max = -13., 13.
Vbg_step_slow = 0.08
Vbg_step_fast = 0.3

inter_delay_slow = 0.5
inter_delay_fast = 0.25

N = 60 #data points
B_start, B_end = 2.5, 6.25
B_crit = 5

for B in np.linspace(B_start, B_end, N):
    #Magnet sweep
    if B>=B_crit:
        persistent_magnet = True
    else:
        persistent_magnet = False
    mag_sweep = SweepIPS(magnet, B, inter_delay=0.25, save_data=False, persistent_magnet=persistent_magnet)
    #mag_sweep.follow_param(ls340.A.temperature)
    
    # Create the database entry
    sample_name = f'B={B:.2f}T_sweep-mag'
    db_entry = DatabaseEntry(db, exp_name, sample_name)
    sweep_queue.append(db_entry)
    sweep_queue.append(mag_sweep)
    
    # TG sweep
    tg_initialize = Sweep1D(keithley_tg.source.voltage, 0, Vtg_min, Vtg_step_fast, inter_delay=inter_delay_fast, x_axis_time=0, save_data = False)
    tg_sweep = Sweep1D(keithley_tg.source.voltage, Vtg_min, Vtg_max, Vtg_step_slow, inter_delay=inter_delay_slow, x_axis_time=0, save_data = True)
    tg_rampdown = Sweep1D(keithley_tg.source.voltage, Vtg_max, 0, Vtg_step_fast, inter_delay=inter_delay_fast, x_axis_time=0, save_data = False)
    
    tg_sweep.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage, sr830.X, sr830.P, sr860.X, sr860.P)
    
    # Create the database entry
    sample_name = f'B={B:.2f}T_Vbg=0V_sweep-Vtg'
    db_entry = DatabaseEntry(db, exp_name, sample_name)
    sweep_queue.append(db_entry)
    sweep_queue.append(tg_initialize, tg_sweep, tg_rampdown)

    bg_initialize = Sweep1D(keithley_bg.source.voltage, 0, Vbg_min, Vbg_step_fast, inter_delay=inter_delay_fast, x_axis_time=0, save_data = False)
    bg_sweep = Sweep1D(keithley_bg.source.voltage, Vbg_min, Vbg_max, Vbg_step_slow, inter_delay=inter_delay_slow, x_axis_time=0, save_data = True)
    bg_rampdown = Sweep1D(keithley_bg.source.voltage, Vbg_max, 0, Vbg_step_fast, inter_delay=inter_delay_fast, x_axis_time=0, save_data = False)
    
    bg_sweep.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage, sr830.X, sr830.P, sr860.X, sr860.P)
    
    # Create the database entry
    sample_name = f'B={B:.2f}T_Vtg=0V_sweep-Vbg'
    db_entry = DatabaseEntry(db, exp_name, sample_name)
    sweep_queue.append(db_entry)
    sweep_queue.append(bg_initialize, bg_sweep, bg_rampdown)
    


In [23]:
for item in sweep_queue.queue:
    print(item)

Database entry saving to C:\Users\nanouser\Documents\MeasureIt\Databases\dw_14-DtBLG.db with experiment name 210203-Landau_fan_unknown_temp_Vsg=10V and sample name B=2.50T_sweep-mag.
Sweeping IPS to 2.5 T.
Database entry saving to C:\Users\nanouser\Documents\MeasureIt\Databases\dw_14-DtBLG.db with experiment name 210203-Landau_fan_unknown_temp_Vsg=10V and sample name B=2.50T_Vbg=0V_sweep-Vtg.
1D Sweep of keithley_tg_source_voltage from 0 to -8.0, with step size -0.2.
1D Sweep of keithley_tg_source_voltage from -8.0 to 8.0, with step size 0.05.
1D Sweep of keithley_tg_source_voltage from 8.0 to 0, with step size -0.2.
Database entry saving to C:\Users\nanouser\Documents\MeasureIt\Databases\dw_14-DtBLG.db with experiment name 210203-Landau_fan_unknown_temp_Vsg=10V and sample name B=2.50T_Vtg=0V_sweep-Vbg.
1D Sweep of keithley_bg_source_voltage from 0 to -13.0, with step size -0.3.
1D Sweep of keithley_bg_source_voltage from -13.0 to 13.0, with step size 0.08.
1D Sweep of keithley_bg_sour

In [24]:
sweep_queue.start()

Starting sweeps
Starting 0D Sweep for 1000000.0 seconds.
Checking the status of the magnet and switch heater.
Heater is already on, so the magnet was not in persistent mode
Done with the sweep, t=1.5599999999976717 s
Done with the sweep!
Finished 0D Sweep of 1000000.0 seconds.
Starting sweep of voltage from 0                           V to -8.0 V
Ramping to our starting setpoint value of 0 V
Sweeping voltage to 0 V
Ramping voltage to 0 . . . 
Done ramping voltage to 0
Sweeping voltage to -8.0 V
Finished sweep of voltage from -8.0                   V to 0 V
Starting sweep of voltage from -8.0                           V to 8.0 V
Ramping to our starting setpoint value of -8.0 V
Done ramping voltage to -8.0
Sweeping voltage to 8.0 V
Starting experimental run with id: 987. 
Finished sweep of voltage from 8.0                   V to -8.0 V
Starting sweep of voltage from 8.0                           V to 0 V
Ramping to our starting setpoint value of 8.0 V
Sweeping voltage to 8.0 V
Ramping vo

In [ ]:
sweep_queue.current_sweep

In [ ]:
print(sweep_queue.current_sweep)

In [ ]:
sweep.update_values()

In [41]:
sweep_queue.current_sweep

False

In [ ]:
sweep.runner

In [ ]:
sweep.save_data

In [15]:
sweep_queue.current_sweep.stop()

In [16]:
magnet.activity(0)

In [16]:
sweep_queue.queue

deque([Save File: (C:\Users\nanouser\Documents\MeasureIt\Databases\dw_14-DtBLG.db, 210129-testing-magnet-sweep, B=0.10T_sweep-mag),
       SweepIPS(0.1 T),
       Save File: (C:\Users\nanouser\Documents\MeasureIt\Databases\dw_14-DtBLG.db, 210129-testing-magnet-sweep, B=0.10T_Vbg=0V_sweep-Vtg),
       Sweep1D(keithley_tg_source_voltage, 0, -2.0, -0.2),
       Sweep1D(keithley_tg_source_voltage, -2.0, 2.0, 0.05),
       Sweep1D(keithley_tg_source_voltage, 2.0, 0, -0.2),
       Save File: (C:\Users\nanouser\Documents\MeasureIt\Databases\dw_14-DtBLG.db, 210129-testing-magnet-sweep, B=0.10T_Vtg=0V_sweep-Vbg),
       Sweep1D(keithley_bg_source_voltage, 0, -2.0, -0.3),
       Sweep1D(keithley_bg_source_voltage, -2.0, 2.0, 0.08),
       Sweep1D(keithley_bg_source_voltage, 2.0, 0, -0.3),
       Save File: (C:\Users\nanouser\Documents\MeasureIt\Databases\dw_14-DtBLG.db, 210129-testing-magnet-sweep, B=0.30T_sweep-mag),
       SweepIPS(0.3 T),
       Save File: (C:\Users\nanouser\Documents\Measure